##ExperimentSetting:OntheimpactoffeaturesorthogonalityforLinkRepresentationLearningwithMessagePassingNeuralNetwork(MPNN)
ThisexperimentaimstoanalyzethecapacityofMPNNstocapturestructuralfeaturesundervaryingconfigurations,includingthetypeofMPNN,thenodefeaturesemployed.


##Trial1

1.SimplifiedMPNN:isamappingfrom$f(\tilde{\mathbf{A}},\mathbf{X})\to\mathbf{H}$,$\text{softmax}(\text{Act}((\mathbf{\tilde{A}XW^0)W^1}))$.
-$\mathbf{h_i}\in\mathbb{R}^{d}$,embeddingofvertex$i$,$n$:numberofvertex,$d$numberofdimension.
-$\mathbf{X}\in\mathbb{R}^{n\timesd}$:initialnodefeatures
-$\mathbf{H}^*$:optimizednodeembeddingw.r.t.all$\mathbf{W}$
-$\text{Act}$:Activationfunction,mostlynonlinearand[ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)).


2.LossFunction:$\argmin_{H^*}\underbrace{\sum_{(i,j)\inE_\text{pos}}\Vert1-\sigma(h_i,h_j)^\top\Vert_2^2}_{\text{positivesamples}}+\underbrace{\sum_{(i,j)\inE_{\text{neg}}}\Vert(0-\sigma(h_i,h_j)^\top\Vert)_2^2}_{\text{negativesamples}}$



In [ ]:
importos
importpandasaspd
importmatplotlib.pyplotasplt
importnumpyasnp
importpandasaspd

root='/pfs/work7/workspace/scratch/cc7738-kdd25/Universal-MP/trials/'

In [1]:
#ListtostoreDataFrames
defmerge_csv_files(results_dir,output_file,key):
dataframes=[]
results_dir=root+results_dir
#Iterateoverallfilesinthedirectory
forfilenameinos.listdir(results_dir):
iffilename.endswith(f"{key}.csv"):#Checkiffileendswith'CN_ddi.csv'
file_path=os.path.join(results_dir,filename)
print(f"Loadingfile:{file_path}")
df=pd.read_csv(file_path)#ReadtheCSVfile
dataframes.append(df)

#ConcatenateallDataFrames
ifdataframes:
merged_ppr=pd.concat(dataframes,ignore_index=True)
#Groupby'Model'and'NodeFeat'andcalculatemeanandvariance
stats=merged_ppr.groupby(['Model','NodeFeat'])['Test_Loss'].agg(['mean','var']).reset_index()

stats.to_csv(output_file,index=False)
print(f"Merged{len(dataframes)}filesinto{output_file}")
else:
print("Nofilesendingwith'PPR_ddi.csv'foundinthedirectory.")
returnmerged_ppr


In [ ]:
#Datavaluesfromthemergedstatistics
defplot_result(df):

#Groupandprocessthedata
result=df.groupby(['Model','NodeFeat'])['Test_Loss'].agg(['mean','var']).reset_index()
result.columns=['Model','NodeFeat','Mean_Test_Loss','Variance_Test_Loss']

#Handlenon-positivevariance
result['Variance_Test_Loss']=result['Variance_Test_Loss'].apply(lambdax:xifx>0else1e-9)

#Uniquemodelsandfeatures
unique_models=result['Model'].unique()
unique_features=result['NodeFeat'].unique()

#Definecolors
colors=['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6']

#Defineplotsettings
width=0.15#Widthofeachbar
x=np.arange(len(unique_models))#Xpositionsforthemodels

#Createtheplot
fig,ax=plt.subplots(figsize=(12,6))

fori,featureinenumerate(unique_features):
feature_data=result[result['NodeFeat']==feature]
values=feature_data['Mean_Test_Loss'].values
errors=np.sqrt(feature_data['Variance_Test_Loss'].values)#Standarddeviationforerrorbars

#Plotbarwitherrorbars
ax.bar(x+i*width,values,width,yerr=errors,label=feature,color=colors[i%len(colors)],capsize=5)

#Setlabelsandtitle
ax.set_ylabel('MeanTestLoss')
ax.set_title('MeanandVarianceofTestLossforCommonNeighborAcrossModels',loc='center',fontsize=14,fontweight='bold')
ax.set_xticks(x+width*(len(unique_features)-1)/2)
ax.set_xticklabels(unique_models)
ax.legend(loc='upperright',title="FeatureType")

plt.tight_layout()
returnplt

In [ ]:
#Definethedirectorycontainingthefiles
results_dir="results/ddi"
output_file="merged_CN_ddi.csv"

merged_cn=merge_csv_files("results/ddi","merged_CN_ddi.csv","CN_ddi")
print(merged_cn)
img=plot_result(merged_cn)
img.savefig('CN_ddi.png')

merged_ppr=merge_csv_files("results/ddi","merged_PPR_ddi.csv","PPR_ddi")
print(merged_ppr)
img=plot_result(merged_ppr)
img.savefig('PPR_ddi.png')

In [ ]:
merged_ppr

In [ ]:
merged_cn

In [ ]:
importmatplotlib.pyplotasplt
importnumpyasnp
importpandasaspd

#Datavaluesfromthemergedstatistics

defplot_result(df):
#Plotsettings
fig,ax=plt.subplots(figsize=(10,6))
x=np.arange(len(df["Model"].unique()))#x-axispositionsforeachmethod
width=0.15#Adjustedwidthforadditionalfeaturetype

colors=['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6']#Softpastelshades

unique_features=df["NodeFeat"].unique()
methods=df["Model"].unique()

complete_index=pd.MultiIndex.from_product([methods,unique_features],names=["Model","NodeFeat"])
df=df.set_index(["Model","NodeFeat"]).reindex(complete_index,fill_value=0).reset_index()

fori,featureinenumerate(unique_features):
feature_data=df[df["NodeFeat"]==feature]
values=feature_data["Mean_Loss"].values
errors=np.sqrt(feature_data["Variance_Loss"].values)#Standarddeviationforerrorbars

#Plotbarwitherrorbars
ax.bar(x+i*width,values,width,yerr=errors,label=feature,color=colors[i],capsize=5)

#Addingdatalabelsformeanvalues
forj,valinenumerate(values):
ax.text(x[j]+i*width,val+0.001,f'{val:.3f}',ha='center',va='bottom')

#Setlabelsandtitle
ax.set_ylabel('MeanTestLoss')
ax.set_title('MeanandVarianceofTestLossforCommonNeighborAcrossModels',loc='center',fontsize=14,fontweight='bold')
ax.set_xticks(x+width*(len(unique_features)-1)/2)
ax.set_xticklabels(methods)
ax.legend(loc='upperright',title="FeatureType")


plt.tight_layout()
plt.show()



###GCN
$\mathbf{X}^{\prime}=\mathbf{\hat{D}}^{-1/2}\mathbf{\hat{A}}
\mathbf{\hat{D}}^{-1/2}\mathbf{X}\mathbf{W}$

$\mathbf{\hat{A}}=\mathbf{A}+\mathbf{I}$denotestheadjacencymatrixwithinsertedself-loopsand
$\hat{D}_{ii}=\sum_{j=0}\hat{A}_{ij}$itsdiagonaldegreematrix.

$\mathbf{x}^{\prime}_i=\mathbf{W}^{\top}\sum_{j\in
\mathcal{N}(i)\cup\{i\}}\frac{e_{j,i}}{\sqrt{\hat{d}_j
\hat{d}_i}}\mathbf{x}_j$

$\hat{d}_i=1+\sum_{j\in\mathcal{N}(i)}e_{j,i}$,where$e_{j,i}$denotestheedgeweightfromsourcenode`j`totargetnode`i`(default:`1.0`)

###SAGE
$\mathbf{x}^{\prime}_i=\mathbf{W}_1\mathbf{x}_i+\mathbf{W}_2\cdot
\mathrm{mean}_{j\in\mathcal{N(i)}}\mathbf{x}_j$


$\mathbf{x}_j\leftarrow\sigma(\mathbf{W}_3\mathbf{x}_j+
\mathbf{b})$

###GIN
$\mathbf{x}^{\prime}_i=h_{\mathbf{\Theta}}\left((1+\epsilon)\cdot
\mathbf{x}_i+\sum_{j\in\mathcal{N}(i)}\mathrm{ReLU}
(\mathbf{x}_j+\mathbf{e}_{j,i})\right)$

###LINKX
$\mathbf{H}_{\mathbf{A}}=\text{MLP}_{\mathbf{A}}(\mathbf{A})$

$\mathbf{H}_{\mathbf{X}}=\textrm{MLP}_{\mathbf{X}}(\mathbf{X})$

$\mathbf{Y}=\textrm{MLP}_{f}\left(\sigma\left(\mathbf{W}
[\mathbf{H}_{\mathbf{A}},\mathbf{H}_{\mathbf{X}}]+
\mathbf{H}_{\mathbf{A}}+\mathbf{H}_{\mathbf{X}}\right)\right)$

In [ ]:
plot_result(merged_ppr)

PPR